# Parâmetros e assinatura de funções

Entender os tipos de parâmetros que aparecem no `help`:

`*args` → múltiplos argumentos posicionais

`**kwargs` → múltiplos argumentos nomeados

`/` → apenas positional-only

`*` → apenas keyword-only

In [ ]:
def f(a, b, /, c, *, d):
    return a + b + c + d

In [ ]:
help(f)

Help on function f in module __main__:

f(a, b, /, c, *, d)



## / → positional-only parameters (somente posição)

Os parâmetros antes da barra / só podem ser passados posicionalmente, não aceitam nomes.

In [ ]:
def f(a, b, /, c):
    return a + b + c

print(f(1, 2, 3))        #  funciona
print(f(1, b=2, c=3))    # erro: b é positional-only

6


TypeError: f() got some positional-only arguments passed as keyword arguments: 'b'

Explicação: a e b só podem ser passados por posição, c pode ser posicional ou nomeado.

## keyword-only parameters (somente nomeados)

Os parâmetros após o asterisco * só podem ser passados com nome.

In [ ]:
def g(a, b, *, c, d):
    return a + b + c + d

print(g(1, 2, c=3, d=4))  # funciona
print(g(1, 2, 3, 4))      # erro: c e d devem ser nomeados

10


TypeError: g() takes 2 positional arguments but 4 were given

Explicação: c e d não aceitam posição, obrigatoriamente usamos c=valor.

## *args → múltiplos argumentos posicionais

In [ ]:
def h(*args):
    return sum(args)

print(h(1,2,3,4))  # 10

10


Explicação: qualquer número de argumentos posicionais é capturado como uma tupla args.

## **kwargs → múltiplos argumentos nomeados

In [ ]:
def i(**kwargs):
    return kwargs

print(i(a=1, b=2))  # {'a': 1, 'b': 2}

{'a': 1, 'b': 2}


Explicação: captura argumentos nomeados em um dicionário kwargs.

# Instrospecção

Introspecção em Python é o ato de inspecionar em tempo de execução o que um objeto é, do que ele é feito e como ele pode ser usado.

Utilidade prática: quando você não sabe exatamente o tipo ou a estrutura interna de uma variável (especialmente em código quebrado, dados aninhados ou APIs externas), a introspecção te dá um raio-x imediato do objeto, permitindo entender e corrigir o problema.

## identidade do objeto

`id(obj)`

Retorna um número único que representa o endereço de memória do objeto.

Útil em maratonas: detectar se duas variáveis apontam para o mesmo objeto, ou se uma operação gerou uma cópia.

In [ ]:
a = [1,2,3]
b = a
c = a[:]

print(id(a))  # ex: 140474986734656
print(id(b))  # mesmo id que a
print(id(c))  # diferente id, cópia da lista

133839576479040
133839576479040
133839576476480


## tipo do objeto

`type(obj)`

Retorna o tipo do objeto.

Útil em maratonas: quando você recebe entrada genérica ou precisa diferenciar tipos.

In [ ]:
print(type(123))       # <class 'int'>
print(type([1,2,3]))   # <class 'list'>
print(type({'a':1}))   # <class 'dict'>

<class 'int'>
<class 'list'>
<class 'dict'>


## checar herança/tipo

`isinstance(obj, Class)`

Retorna True se obj for instância de Class ou de subclasses.

Útil: mais seguro que type() quando subclasses podem aparecer.

In [1]:
print(isinstance([1,2,3], list))    # True
print(isinstance((1,2), tuple))     # True
print(isinstance({1,2}, set))       # True

True
True
True


## Estruturas Aninhadas

In [4]:
data = [
    {"nome": "Ana", "notas": [5, 8, 7]},
    {"nome": "Bianca", "notas": [8, 9, 10]},
    {"nome": "Carlos", "notas": [4, 6, 7]},
]

# introspecção com list comp → tipo de cada "notas"
tipos = [ (aluno["nome"], [type(n) for n in aluno["notas"]]) for aluno in data ]
print(tipos)

# Utilizando for
print()
for aluno in data:
    print(aluno["nome"], type(aluno["notas"]), [type(n) for n in aluno["notas"]])

[('Ana', [<class 'int'>, <class 'int'>, <class 'int'>]), ('Bianca', [<class 'int'>, <class 'int'>, <class 'int'>]), ('Carlos', [<class 'int'>, <class 'int'>, <class 'int'>])]

Ana <class 'list'> [<class 'int'>, <class 'int'>, <class 'int'>]
Bianca <class 'list'> [<class 'int'>, <class 'int'>, <class 'int'>]
Carlos <class 'list'> [<class 'int'>, <class 'int'>, <class 'int'>]


# Helper

O debug_var é um helper de introspecção que mostra, de forma organizada:

🔎 Tipo e id da variável

📦 Estruturas (listas, tuplas, sets, dicionários) com tamanho e tipos internos

🔁 Exploração recursiva em estruturas aninhadas

🧩 Objetos customizados, exibindo a classe e os atributos disponíveis

👉 Em resumo: ele responde rapidamente “o que é essa variável e do que ela é feita?”, ajudando no debug quando você se perde no meio de estruturas ou objetos.

In [5]:
def debug_var(x, name="var", level=0):
    """Helper para introspecção de variáveis em Python."""
    indent = "   " * level
    print(f"{indent} {name} -> type: {type(x)}, id: {id(x)}")

    # Casos simples (int, str, float, bool, None, etc.)
    if isinstance(x, (int, float, str, bool, type(None))):
        print(f"{indent}   Valor direto: {repr(x)}")

    # Estruturas iteráveis
    elif isinstance(x, (list, tuple, set, frozenset)):
        print(f"{indent}   Estrutura {type(x).__name__} com {len(x)} elementos")
        if len(x) > 0:
            print(f"{indent}   Tipos internos: {[type(e) for e in x]}")
            # introspecção recursiva (um nível a mais)
            for i, e in enumerate(x):
                debug_var(e, name=f"{name}[{i}]", level=level+1)

    # Dicionários
    elif isinstance(x, dict):
        print(f"{indent}   Dicionário com {len(x)} pares")
        if len(x) > 0:
            print(f"{indent}   Tipos de chaves: {[type(k) for k in x.keys()]}")
            print(f"{indent}   Tipos de valores: {[type(v) for v in x.values()]}")
            for k, v in x.items():
                debug_var(v, name=f"{name}[{repr(k)}]", level=level+1)

    # Objetos customizados
    else:
        print(f"{indent}   Objeto da classe: {x.__class__.__name__}")
        print(f"{indent}   Atributos disponíveis: {dir(x)}")

In [6]:
# Exemplo 1: número simples
debug_var(42, "idade")

# Exemplo 2: lista aninhada
dados = [1, [2, 3], {"a": 4, "b": [5, 6]}]
debug_var(dados, "dados")

# Exemplo 3: dicionário simples
info = {"nome": "Ana", "idade": 30}
debug_var(info, "info")

# Exemplo 4: objeto customizado
class Pessoa:
    def __init__(self, nome, idade):
        self.nome = nome
        self.idade = idade

p = Pessoa("João", 25)
debug_var(p, "pessoa")

 idade -> type: <class 'int'>, id: 11643400
   Valor direto: 42
 dados -> type: <class 'list'>, id: 137409060453568
   Estrutura list com 3 elementos
   Tipos internos: [<class 'int'>, <class 'list'>, <class 'dict'>]
    dados[0] -> type: <class 'int'>, id: 11642088
      Valor direto: 1
    dados[1] -> type: <class 'list'>, id: 137409063290944
      Estrutura list com 2 elementos
      Tipos internos: [<class 'int'>, <class 'int'>]
       dados[1][0] -> type: <class 'int'>, id: 11642120
         Valor direto: 2
       dados[1][1] -> type: <class 'int'>, id: 11642152
         Valor direto: 3
    dados[2] -> type: <class 'dict'>, id: 137409060547200
      Dicionário com 2 pares
      Tipos de chaves: [<class 'str'>, <class 'str'>]
      Tipos de valores: [<class 'int'>, <class 'list'>]
       dados[2]['a'] -> type: <class 'int'>, id: 11642184
         Valor direto: 4
       dados[2]['b'] -> type: <class 'list'>, id: 137409060447104
         Estrutura list com 2 elementos
         Tipos 